In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Makeover Monday 2020 W6 –– US Wartime Living

For this week's challenge, we are looking at the percentage of lifetime according to birth year where the USA was at war.

Source Article: [Nearly a quarter of Americans have never experienced the U.S. in a time of peace](https://www.washingtonpost.com/politics/2020/01/08/nearly-quarter-americans-have-never-experienced-us-time-peace/)

Data Source: [Washington Post](https://www.washingtonpost.com/)

Makeover Monday Data: [data.world](https://data.world/makeovermonday/2020w6)

## Data Analysis

### Loading the Data:

In [16]:
df = pd.read_excel("US life at war.xlsx")

### Summary Statistics:

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 2 columns):
Birth year                               115 non-null int64
% of your life the US has been at war    115 non-null float64
dtypes: float64(1), int64(1)
memory usage: 1.9 KB


In [18]:
df.describe()

,Birth year,% of your life the US has been at war
count,115.000000,115.000000
mean,1962.000000,0.574635
std,33.341666,0.221140
min,1905.000000,0.348000
25%,1933.500000,0.436000
50%,1962.000000,0.471000
75%,1990.500000,0.673000
max,2019.000000,1.000000


In [19]:
df.head()

,Birth year,% of your life the US has been at war
0,1905,0.348
1,1906,0.351
2,1907,0.354
3,1908,0.357
4,1909,0.360
